In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, 7),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Dropout(0.25),
            nn.Conv2d(32, 40, 3),
            nn.ReLU(),
            nn.BatchNorm2d(40),
            # nn.MaxPool2d(kernel_size = 2, stride = 2),
            nn.Dropout(0.25),
            nn.Conv2d(40, 128, 1),
            nn.ReLU(),
            # nn.BatchNorm2d(128),
            # nn.Dropout(0.25)
        )
        self.fc1 = nn.Linear(128, 10)
        # self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(F.adaptive_avg_pool2d(x, (1, 1)))
        x = x.view(-1, 128)
        # x = F.relu(self.fc1(x))
        x = self.fc1(x)
        return F.log_softmax(x)

In [ ]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 22, 22]           1,600
              ReLU-2           [-1, 32, 22, 22]               0
       BatchNorm2d-3           [-1, 32, 22, 22]              64
         MaxPool2d-4           [-1, 32, 11, 11]               0
           Dropout-5           [-1, 32, 11, 11]               0
            Conv2d-6             [-1, 40, 9, 9]          11,560
              ReLU-7             [-1, 40, 9, 9]               0
       BatchNorm2d-8             [-1, 40, 9, 9]              80
           Dropout-9             [-1, 40, 9, 9]               0
           Conv2d-10            [-1, 128, 9, 9]           5,248
             ReLU-11            [-1, 128, 9, 9]               0
           Linear-12                   [-1, 10]           1,290
Total params: 19,842
Trainable params: 19,842
Non-trainable params: 0
---------------------------------

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        # transforms.Normalize((0.485,), (0.229,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                        # transforms.Normalize((0.485,), (0.229,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct_train = 0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct_train += pred.eq(target.view_as(pred)).sum().item()
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        loss, correct_train, len(train_loader.dataset),
        100. * correct_train / len(train_loader.dataset)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

for epoch in range(20):
    print("epoch:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

epoch: 0


loss=0.11146663874387741 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.91it/s]


Train set: Average loss: 0.1115, Accuracy: 54886/60000 (91.48%)




Test set: Average loss: 0.1033, Accuracy: 9678/10000 (96.78%)

epoch: 1


loss=0.01885143853724003 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.82it/s]

...


Train set: Average loss: 0.0302, Accuracy: 59271/60000 (98.78%)




Test set: Average loss: 0.0283, Accuracy: 9904/10000 (99.04%)

epoch: 19


loss=0.02028738334774971 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 21.04it/s]


Train set: Average loss: 0.0203, Accuracy: 59311/60000 (98.85%)




Test set: Average loss: 0.0293, Accuracy: 9900/10000 (99.00%)

